In [2]:
import os
import pandas as pd
import numpy as np
import xarray as xr
import datetime
import matplotlib as plt

In [3]:
import sys
# setting path
sys.path.append('../')

from ruins.processing import drain_cap

In [4]:
# load primary data
data = xr.load_dataset('./data/hydro_krummh.nc').to_dataframe()
data.head()

,Prec_dissagg_1,Prec_dissagg_2,Prec_dissagg_3,Prec_dissagg_4,Prec_dissagg_5,Prec_dissagg_6,Prec_dissagg_7,Prec_dissagg_8,Prec_dissagg_9,Prec_dissagg_10,...,Prec_dissagg_77,Prec_dissagg_78,Prec_dissagg_79,Prec_dissagg_80,Prec,wl_Knock_Outer,wl_LW,wl_Knock_Inner,Knock_pump_obs,Q_Knock
date,,,,,,,,,,,,,,,,,,,,,
2011-01-01 00:00:00,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.1,NaN,-1325.000000,NaN,NaN,NaN
2011-01-01 01:00:00,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.1,NaN,-1325.000000,NaN,NaN,NaN
2011-01-01 02:00:00,0.0,0.0,0.0,0.491553,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,NaN,-1319.090942,NaN,NaN,NaN
2011-01-01 03:00:00,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.1,NaN,-1315.000000,NaN,NaN,NaN
2011-01-01 04:00:00,0.0,0.0,0.0,0.038447,0.0,0.240214,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.1,NaN,-1310.877563,NaN,NaN,NaN


In [7]:
result = pd.DataFrame(columns = ['max_store', 'energy_cons', 'rcp', 'SLR','prec_incr','event','canal_para', 'canal_area','forecast_pump','raindissagg'])
i=0
for RCP in ['rcp26','rcp45','rcp85']:
    print(RCP)
#for RCP in ['rcp26']:
    for ClimateTime in ['2065']:
        if RCP == 'rcp26':
            if ClimateTime == '2010':
                SLRs = [35,122,221]  # SLR in 2030 SSP126, outer range of low/medium confidence
                prec_increases = [0.9, 1, 1.1]
            if ClimateTime == '2065':
                SLRs = [154, 432, 848] # SLR in 2090 SSP126, outer range of low/medium confidence
                prec_increases = [0.8, 0.9, 1, 1.1, 1.2]
        if RCP == 'rcp45':
            if ClimateTime == '2010':
                SLRs = [29,126,232]  # SLR in 2030 SSP245
                prec_increases = [0.9, 1, 1.1]
            if ClimateTime == '2065':
                SLRs = [249, 522, 918] # SLR in 2090 SSP245
                prec_increases = [0.8, 0.9, 1, 1.1, 1.2]
        if RCP == 'rcp85':
            if ClimateTime == '2010':
                SLRs = [34,125,231]  # SLR in 2030 SSP126, outer range of low/medium confidence
                prec_increases = [0.9, 1, 1.1]
            if ClimateTime == '2065':
                SLRs = [379, 730, 1143, 1676] # SLR in 2090 SSP126, lower range of low/medium confidence, upper range: 1143 for medium confidence,1676 for low confidence 
                prec_increases = [0.8, 0.9, 1, 1.1, 1.2, 1.3]
        for prec_increase in prec_increases:
            for slr in SLRs:
                print(slr)
####                for rainshift in [-12,-6,0,6,12]:
                for raindissagg in data.filter(regex='Prec').columns:
                    for event in ['2012','2017-03','2018-01','2017-12','2017-02','2011']:
                        if event == "2012":
                            t1 = datetime.date(2011, 12, 29)
                            t2 = datetime.date(2012, 1, 12)
####                            hourly_recharge = weather_1h["Prec"].shift(rainshift)[t1:t2]
                            hourly_recharge = data[raindissagg][t1:t2]

                        if event == "2017-03":
                            t1 = datetime.date(2017, 3, 15)
                            t2 = datetime.date(2017, 3, 29)
####                            hourly_recharge = weather_1h["Prec"].shift(rainshift)[t1:t2] * 0.9   # calibration to account for ?not fully saturated soils?
                            hourly_recharge = data[raindissagg][t1:t2]
                            
                        if event == "2018-01":
                            t1 = datetime.date(2017, 12, 30)
                            t2 = datetime.date(2018, 1, 13)
####                            hourly_recharge = weather_1h["Prec"].shift(rainshift)[t1:t2]
                            hourly_recharge = data[raindissagg][t1:t2]

                        if event == "2017-12":
                            t1 = datetime.date(2017, 12, 4)
                            t2 = datetime.date(2017, 12, 18)
####                            hourly_recharge = weather_1h["Prec"].shift(rainshift)[t1:t2]
                            hourly_recharge = data[raindissagg][t1:t2]

                        if event == "2017-02":
                            t1 = datetime.date(2017, 2, 19)
                            t2 = datetime.date(2017, 3, 5)
####                            hourly_recharge = weather_1h["Prec"].shift(rainshift)[t1:t2] * 0.9   # calibration to account for ?not fully saturated soils?
                            hourly_recharge = data[raindissagg][t1:t2]

                        if event == "2011":
                            t1 = datetime.date(2011, 12, 3)
                            t2 = datetime.date(2011, 12, 17)
####                            hourly_recharge = weather_1h["Prec"].shift(rainshift)[t1:t2]
                            hourly_recharge = data[raindissagg][t1:t2]
        

                        days = (t2-t1).days
                        hours = days*24 + 1

                        tide = data['wl_Knock_Outer'][t1:t2] + slr     # add SLR to tide water level [mm]

                        hourly_recharge = hourly_recharge.rolling("12h").mean() # smoothen rainfall signal to account for water redistribution within catchment
                        hourly_recharge *= prec_increase  # scale hourly recharge by change in climate forcing

                        for advance_pump in [0,50]:
                            for Canal_area in [4,6]:
#                        for advance_pump in [0]:
#                            for Canal_area in [4]:

                                maxdh = 4000     # set maximum pump gradient [mm]
                                wig = tide*0     # no wind induced gradient -> this may be used to simulate a local flooding in a subcatchment

                                canal_par = [(1.112 , 4156.), (0.9946, 2142.), (1.045 , 2820.)]

                                x = pd.DataFrame.from_dict(
                                    {        
                                        'recharge' : hourly_recharge,
                                        'h_tide' : tide,
                                        'wig' : wig,
                                    }
                                )
                                for z in canal_par:
#                                    if i < breaked:
#                                        i += 1
#                                        next
#                                    else:
    #                                z[1] /= canal_flow_scale
                                    x['h_store'], q_pump, x['h_min'], x['flow_rec'], pump_cost, x['store'] = drain_cap.storage_model(forcing_data = x,
                                                                                                                                     canal_par = z,
                                                                                                                                     v_store = 0,
                                                                                                                                     h_store_target = -1350,
                                                                                                                                     canal_area = Canal_area,
                                                                                                                                     h_forecast_pump = advance_pump,
                                                                                                                                     h_grad_pump_max = maxdh)
    #                                my_dict[RCP+'_slr' + str(slr) + '_prec-inc' + str(prec_increase) + '_event' + event + '_adv-pump' + str(advance_pump) + '_canalarea' + str(Canal_area) + '_canalparameter' + str(z)] = max(x['store'])
                                    i += 1
####                                    result.loc[str(i),:] = [max(x['store']), sum(pump_cost), RCP, slr,prec_increase,event,str(z),Canal_area, advance_pump, rainshift]
                                    result.loc[str(i),:] = [max(x['store']), sum(pump_cost), RCP, slr,prec_increase,event,str(z),Canal_area, advance_pump, raindissagg]

result.to_csv("new_try"+datetime.datetime.now().strftime("%Y%m%d%H%M%S")+".csv")
result

rcp26
154
432
848
154
432
848
154
432
848
154
432
848
154
432
848
rcp45
249
522
918
249
522
918
249
522
918
249
522
918
249
522
918
rcp85
379
730
1143
1676
379
730
1143
1676
379
730
1143
1676
379
730
1143
1676
379
730
1143
1676
379
730
1143
1676


,max_store,energy_cons,rcp,SLR,prec_incr,event,canal_para,canal_area,forecast_pump,raindissagg
1,43.1134,59.6669,rcp26,154,0.8,2012,"(1.112, 4156.0)",4,0,Prec_dissagg_1
2,46.4329,52.011,rcp26,154,0.8,2012,"(0.9946, 2142.0)",4,0,Prec_dissagg_1
3,44.6805,55.8371,rcp26,154,0.8,2012,"(1.045, 2820.0)",4,0,Prec_dissagg_1
4,44.0507,58.5321,rcp26,154,0.8,2012,"(1.112, 4156.0)",6,0,Prec_dissagg_1
5,47.3352,50.9307,rcp26,154,0.8,2012,"(0.9946, 2142.0)",6,0,Prec_dissagg_1
...,...,...,...,...,...,...,...,...,...,...
314924,100.977,50.2313,rcp85,1676,1.3,2011,"(0.9946, 2142.0)",4,50,Prec
314925,98.7516,54.096,rcp85,1676,1.3,2011,"(1.045, 2820.0)",4,50,Prec
314926,97.3921,57.0806,rcp85,1676,1.3,2011,"(1.112, 4156.0)",6,50,Prec
314927,101.825,49.2158,rcp85,1676,1.3,2011,"(0.9946, 2142.0)",6,50,Prec
